# Hackaton Würzburg 2025
## Hack and Heal
- Teammitglieder: Lukas, Rafael, Katharina 
- Technischer Mentor: Daniel

### Die Challenge
Worum gehts? 
- Livedovaskulopathie als seltene und oft fehldiagnostizierte Erkrankung 
Was sollen wir tun? 
- Training eines Algorithmus zur Unterscheidung von Livedovaskulopathie und anderen Wunden 
- Basis anhand von Fotoaufnahmen aus der Hautklinik Würzburg

Langfristiges Ziel? 
- Einbinden in bestehende Wund-App der Hautklink Würzburg als Entscheidungshilfe für ÄrztInnen

## Step 1.1 Daten vorverarbeiten
Daten sichten:
1825 Bilder, davon
- 557 Bilder zu Livdeovaskulopathie
- 1268 Bilder anderer Wunden

Ziel: Bilder vom Trainings- und Testdaten in einheitlichem Format und Reduktion von Artefakten

Vorgehen:
- Händisches Entfernen der Daten aus dem Ordner der Livedovaskulopathie
- Kriterien:
  - Es muss eine Wunde vorhanden sein (Gefahr der Erkennung, dass bei Livedovaskulopathie häufig geschlossene Haut, bzw v.a. die Livedozeichnung, fotografiert wurde)
  - Nur Bilder vom Bein (Gefahr der Erkennung, dass bei Livedovaskulopathie häufiger Wunden am Unterschenkel sind als im anderen Ordner)

Einteilen des ersten Datensatzes in Trainingsdatensatz und Testdatensatz 80:20
- Training LV (Livedovaskulopathie): 166, 1047 Nicht-LV
- Test LV: 29, 222 Nicht-LV


## Step 1.2 Training des Neuronalen Netzwerks: Erster Lauf
- Verwendung des AutoML-Frameworks AucMedi, basierend auf CNN DenseNet121
- Verwendung von Grad-CAM als ExplainableAI

```bash
aucmedi training \
    --path_imagedir ./data/training \
    --path_modeldir ./training_1/model \
    --epochs 500
```

Ergebnis
- Model stoppt nach 23 Epochen da in den letzten 12 Epochen Loss function nicht besser wurde
- Model gibt Wahrscheinlichkeiten aus
- Decision Threshold für Klassifizuerung auf 0,5 (üblicher Wert in der Literatur) gesetzt
- Auswertung der Qualität des Modells anhand der zurseite gelegten Test-Daten: 

|                      | Tatsächlich positiv | Tatsächlich negativ |
|----------------------|---------------------|----------------------|
| **Vorhergesagt positiv** | True Positive: 29     | False Positive: 31     |
| **Vorhergesagt negativ** | False Negative: 0     | True Negative: 191     |

  + Berechnung F1-Score: 0.65
  + Sensitivität: 1.0

## Step 1.3 Auswertung des trainierten Modells inkl. GradCam-Visualisierung
Dann: Einfärbung der klassifizierten Bilder mit Hilfe von GradCAM (Gradient-weitghted Class Activation Mapping)
-> Sichtbarmachung, welche Bildbrereiche das Neuronale Netz bei der Klassifikatoin besonders berücksichtigt hat (explainable AI)

```bash
aucmedi prediciton \
--path_modeldir ./training_1/model \
--path_imagedir ./data/test/LV_AND_NOT_LV \
--path_pred ./training_1/preds.csv \
--xai_method gradcam \
--xai_directory ./training_1/waermebilder
```
 Ergebnis:
 - Es wurde in ein paar Fällen neben dem Hauptmerkmal (Wunde) auch Objekte wie Maßlineal oder Handschuh im Bild stark berücksichtigt, das v.a. im Ordner der nicht-LV-Wunden häufiger vorkommt.

## Step 2 Erneute Datenvorerarbeitung und neuer Trainingsversuch

### Step 2.1. Daten per Skript bereinigen
- automatisiertes Zuschneiden der Bilder

In [ ]:
import pandas as pd
from pathlib import Path
from PIL import Image
from tqdm import tqdm

wound_data = pd.read_csv('/Volumes/teddoww/UlceraScan/Koordinaten_Segmentierte_Wunden.csv')

clean_wound_data = wound_data.drop(columns=['id', 'image_id', 'hash', 'comment', 'updated_at'])
clean_wound_data = clean_wound_data[clean_wound_data['label'] == 'Wound']
clean_wound_data = clean_wound_data[clean_wound_data['source'] == 'manual']
clean_wound_data = clean_wound_data.drop(columns=['source', 'label'])
clean_wound_data['image_path'] = clean_wound_data['image_path'].str.split('/').str[-1]
clean_wound_data.set_index('image_path', inplace=True)

def handle_single_label(center_x: float, center_y: float, file_path: Path, save_path: Path):
    img = Image.open(file_path)
    width, height = img.size

    save_path = save_path / file_path.parent.name / (file_path.stem + '.png')

    # Convert relative center to pixel coordinates
    cx = int(center_x * width)
    cy = int(center_y * height)

    crop_size = 1024
    half_crop = crop_size // 2

    # Initial crop box
    left = cx - half_crop
    upper = cy - half_crop
    right = cx + half_crop
    lower = cy + half_crop

    # Check if crop fits inside image bounds
    if left >= 0 and upper >= 0 and right <= width and lower <= height:
        crop_box = (left, upper, right, lower)
        cropped = img.crop(crop_box)
        cropped.save(save_path)
        return

    # If none of the above fit directly, try shifting the 1024x1024 crop into bounds
    left = max(0, min(cx - half_crop, width - crop_size))
    upper = max(0, min(cy - half_crop, height - crop_size))
    right = left + crop_size
    lower = upper + crop_size

    crop_box = (left, upper, right, lower)
    cropped = img.crop(crop_box)
    cropped.save(save_path)

def handle_multi_label(df: pd.DataFrame, file_path: Path, save_path: Path):
    xmin = df['xmin'].min()
    xmax = df['xmax'].max()
    ymin = df['ymin'].min()
    ymax = df['ymax'].max()

    center_x = (xmin + xmax) / 2
    center_y = (ymin + ymax) / 2

    handle_single_label(
        center_x,
        center_y,
        file_path,
        save_path
    )

def lockup_image(image_path, save_path):
    try:
        wound_data = clean_wound_data.loc[image_path.name]
    except KeyError:
        return None

    if type(wound_data) is pd.DataFrame:
        handle_multi_label(wound_data, image_path, save_path)
    else:
        handle_single_label(
            (wound_data.xmax + wound_data.xmin) / 2,
            (wound_data.ymax + wound_data.ymin) / 2,
            image_path,
            save_path
        )

def crop_img_data(base_path, save_path):
    for class_name in ["LV", "NOT_LV"]:
        class_path = base_path / class_name
        for file_path in tqdm(class_path.iterdir(), desc=f'Cropping {class_path}'):
            if file_path.is_file():
                lockup_image(file_path, save_path)

def init_dirs(save_path: Path):
    save_path.mkdir(exist_ok=True)

    for class_name in ["LV", "NOT_LV"]:
        (save_path / class_name).mkdir(exist_ok=True)

base_path = Path('/Volumes/teddoww/UlceraScan/')

clean_test_path = base_path / 'Clean_Test_Data'
clean_train_path = base_path / 'Clean_Training_Data'

noisy_test_path = base_path / 'Test_Data'
noisy_train_path = base_path / 'Training_Data'

init_dirs(clean_test_path)
init_dirs(clean_train_path)

crop_img_data(noisy_test_path, clean_test_path)
#crop_img_data(noisy_train_path, clean_train_path)

### Disclaimer: Code from Mentor Daniel



### 2.2 Modell erneut triainieren
- Verwendung des AutoML-Frameworks AucMedi
```bash
aucmedi training \
    --path_imagedir ./data/cropped/training \
    --path_modeldir ./training_2/model \
    --epochs 500
```

Ergebnis
- Stopp nach 42 Epochen
- Decision Threshold auf 0,5 gesetzt

### 2.3 Modell auswerten
```bash
aucmedi prediciton \
--path_modeldir ./training_2/model \
--path_imagedir ./data/cropped/test/LV_AND_NOT_LV \
--path_pred ./training_2/preds.csv \
--xai_method gradcam \
--xai_directory ./training_2/waermebilder
```



In [ ]:
###################
# MODELAUSWERTUNG #
###################

from pathlib import Path
import pandas as pd
ROOT = Path(".")
results_dir = ROOT / "training_2"
data_dir = ROOT / "data" / "cropped"

# determine which samples should have been labeled as "lv" through filenames in test/LV
actual_lv_samples = [f.stem for f in (data_dir / "test" / "LV").iterdir()]

df = pd.read_csv(results_dir / "preds.csv")
df["pred_lv"] = df["LV"] > 0.5
df["actual_lv"] = df["SAMPLE"].map(
    lambda x: x in actual_lv_samples
)
df = df.drop(columns=["LV", "NOT_LV"])
#
TP = ( df.pred_lv &  df.actual_lv).sum()
TN = (~df.pred_lv & ~df.actual_lv).sum()
FP = ( df.pred_lv & ~df.actual_lv).sum()
FN = (~df.pred_lv &  df.actual_lv).sum()

#calculate F1 score
f1_score = 2*TP / (2*TP + FP + FN) # 0.67
f1_score

np.float64(0.6666666666666666)

F1 score beträgt 0,67.

In [ ]:
### Erstellung der Konfusionsmatrix
confusion_matrix = pd.DataFrame(
    {
        "actual_true": [TP, FN],
        "actual_false": [FP, TN],
    },
    index = ["pred_true", "pred_false"]
)
confusion_matrix

,actual_true,actual_false
pred_true,4,2
pred_false,2,20


In [ ]:
### Gegenüberstellung Vorhersagen vs tatsächliches Label (Ausschnitt)
df.head()

,SAMPLE,pred_lv,actual_lv
0,3716_17_ulc025_14112017_2,False,False
1,3909_15_ulc033_21122015_3,False,False
2,4346_20_43_05102020_4,False,False
3,4455_18_37_11102018_5,False,False
4,4545_22_ulc002_14112022_4,False,False


### 4 Fazit
- Wir haben ein Modell trainiert, welches zur automatisierten Erkennung von Livedovaskulopathie genutzt werden kann.

#### Problem der Grunddaten
- **Bildartefakte** wie Hintergrundobjekte, Lineale oder Handschuhe traten im ersten Datensatz häufiger bei einer bestimmten Erkrankungsart auf.  
  → Gefahr: Das Modell könnte diese Artefakte als krankheitsspezifische Merkmale fehlinterpretieren.  
  → Im zweiten Training wurden diese Artefakte weitgehend entfernt, was zu realistischeren Lernergebnissen führt.

- **Wundgröße** zeigt eine Korrelation mit der Diagnose "LV vs. Nicht-LV"
  → Möglicher Bias, wenn das Modell Größe als Hauptunterscheidungsmerkmal nutzt.

#### Vergleich der beiden Trainings
| **Training**        | **Sensitivität** | **Spezifität** | **F1-Score** |
|---------------------|------------------|----------------|--------------|
| Erstes Training     | 100 %            | 89 %           | 0,65         |
| Zweites Training    | 67 %             | 91 %           | 0,67         |



Interpretation der Ergebnisse
- Der Ergebnis-Score hat sich im zweiten Training theoretisch verschlechtert, insbesondere die Sensitivität sank von 100 % auf 67 %.  
  → Auf den ersten Blick könnte man vermuten, dass der ursprüngliche Datensatz besser geeignet gewesen wäre.  
  → **Diese Schlussfolgerung wäre jedoch irreführend**, da:
  - Overfitting auf den Trainingsdatensatz 
  - Das Modell im ersten Training möglicherweise noch deutlicher Artefakte im Bild (z. B. Handschuhe, Lineale, Hintergrund) als relevante Merkmale gelernt hat – ein klassischer Fall von Overfitting auf irrelevante Bildinformationen.

- Zudem: unklar, wie das Modell mit Grenzfällen umgeht, da der Datensatz hauptsächlich klar diagnostizierte Fälle enthielt.
